In [ ]:
import json
import time
from tqdm import trange
from lib.CogscalingLib import Orchestrator

#sessionFile is used for recording or replaying states for offline learning.
sessionFile="record_session-last.h5"
environment = Orchestrator(debug=False, sessionFile=sessionFile)


## Setup

In [ ]:
#Environment setup
Min_front = 1
Max_front = 10
prometheus_trace_number = 12 #Prometheus thread buffer size
prometheus_trace_before_scaling = 0 #Number of prometheus measure to take before raising the event scaling done
WaitForKubernetesWhenScaling = True
WaitForPrometheusWhenScaling = True
zipkin_trace_number = 12000 #Zipkin thread buffer size
zipkin_lookback = 12 #Zipkin thread lookback window -> 10s
MeasureNumberForEachState = 20 #Number of measures to take for each couple [injected user number; number of container]

#Injector setup
min_load = 5
max_load = 10

## Orchestrator configuration and functions

In [ ]:
status, message = environment.setDeploymentLimit(deployment="front-dynamic-component", min=Min_front, max=Max_front)
print("message:", message)

status, message = environment.setZipkinService(services=['istio-ingressgateway','front-dynamic-component.default'])
print("message:", message)


status, message = environment.setPrometheusService(deployments=['front-dynamic-component'])
print("message:", message)


status, message = environment.changePrometheusTraceNumber(trace_number=prometheus_trace_number)
print("message:", message)

status, message = environment.setSampleNumberBeforeAcceptScaling(sampleNumber=prometheus_trace_before_scaling)
print("message:", message)


status, message = environment.changeZipkinTraceNumber(trace_number=zipkin_trace_number)
print("message:", message)

status, message = environment.setZipkinLookback(lookback=zipkin_lookback)
print("message:", message)

status, message = environment.getConfig()
print("configuration:", message)

def changeInjector(user_level):
    status, message = environment.setLocustUser(user=int(user_level),spawn_rate=1)
    print("message:", message)

def ChangeScaling(front_number):
    status, message = environment.setKubernetesDeploymentScale(deployment="front-dynamic-component",number=front_number,waitKubernetes=WaitForKubernetesWhenScaling,waitPrometheus=WaitForPrometheusWhenScaling)
    print("message:", message)

def take_measures(measureCount):
    for num_samples in trange(measureCount):
            #state = environment.getAgregatedState(components=[{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'}],zipkinOnly=False,prometheusOnly=False,record=True,replay=False)
            state = environment.getAgregatedState(components=[{'prometheus': 'front-dynamic-component','zipkin': 'front-dynamic-component-service.default.svc.cluster.local:80/*'}],replay=False, record=True,useMetricServer = False)

## Start capture by increasing user 5 by 5.

In [ ]:
print("Ascent measures front...")
for load in trange(min_load,max_load + 1,5):
    print("Change injector user number:", load)
    changeInjector(load)
    for front_number in trange(Min_front,Max_front + 1):
        print("Change system scaling to front number:", front_number)
        ChangeScaling(front_number)
        take_measures(MeasureNumberForEachState)

print("Descent measures front...")
for load in trange(max_load,0,-5):
    print("Change injector user number:", load)
    changeInjector(load)
    for front_number in trange(Max_front,Min_front -1,-1):
        print("Change system scaling to front number:", front_number)
        ChangeScaling(front_number)
        take_measures(MeasureNumberForEachState)
print("dataset is now ready for offline learning")